# Introduction

## An HTML Layout

```html
<html>
    <body>
        <div>
            stuff
        </div>
        <div>
            more stuff
        </div>
    </body>
</html>
```


## Excelbird Layout

```python
Book(  # no need to create a variable for it.
    Sheet(
        Frame(  # Columnar dataframe
            Col(
                Cell(1),
                Cell(2),
            ),
        ),
        Stack( # Unstructured horizontal container
            Cell(1),
            Row(1, 2, 3),
            Col(5, 6, 7),
            VFrame([[10,20], [30, 40]]),
        )
    )
).write(path)
```

---

## Writing your first `Book`

In [27]:
import excelbird as xb
PATH = "test.xlsx"

You don't care which letter of the alphabet your column is placed in, and in excelbird, your code doesn't either.

A layout is a nested arrangement of elements. Each element will arrange its children
in a different way. This way, you can re-arrange elements by simply re-arranging your code,
without editing it.

Every layout is built inside a `Book`. There's no need to assign it to a variable, since there's
nothing you can do to it except call `.write()`.

`Book` doesn't care what you put inside it. It will just put **each argument into a separate sheet**

In [ ]:
xb.Book(
    xb.Cell(1),  # sheet1
    xb.Cell(2)   # sheet2
).write(PATH)

<img src="https://i.imgur.com/ZjiAObH.png" width="350"/>

A `Sheet` is a container that arranges its elements **vertically**.

In [ ]:
xb.Book(
    xb.Sheet(
        xb.Cell(1),
        xb.Cell(2),
    )
).write(PATH)

<img src="https://i.imgur.com/0ptIppO.png" width="350"/>

A `Col` is also a vertical container, but it's *structured*, and contains **Cells only**

In [ ]:
# 'import *' should usually be avoided, but it'll make this demo more readable
from excelbird import *

Book(
    Sheet(
        Col(Cell(1), Cell(2)),
        Col(1, 2)  # Any value will be converted to Cell
    )
).write(PATH)

<img src="https://i.imgur.com/CE1i2Ag.png" width="350"/>

That's not quite right. We need a container that's **horizontally arranged**.

There are two options: `Stack` (unstructured), and `Frame` (structured).
We'll talk more about the differences later, but *if possible*, always use a Frame for holding Cols

In [ ]:
Book(
    Sheet(
        Frame(
            Col(Cell(1), Cell(2)),
            Col(1, 2)  # Any value will be converted to Cell
        )
    )
).write(PATH)

<img src="https://i.imgur.com/70unHDv.png" width="350"/>

All containers (besides Book and Sheet) have a **sibling type**: an object that functionally identical,
but arranges its children along the opposite axis

`Col`'s sibling type is `Row`

In [ ]:
Book(
    Sheet(
        Frame(
            Col(Cell(1), Cell(2)),
            Col(1, 2)
        ),
        Row(10, 20, 30), # <-------
    )
).write(PATH)

<img src="https://i.imgur.com/CHthDUs.png" width="350"/>

## Styling

This is one of the greatest advantages of using a tree-like layout design instead of scripting.
If you have experience in HTML/CSS the following examples will feel intuitive. Here are the rules:

1. A container's styling will be passed down to each of its children
2. An element will **always** override its parent for each style attribute declared directly.
3. Exceptions to rule #1 are made when the styling is *spatial* in nature. For instance, ``border`` will always be applied to the perimeter of the element it was declared on.

Before we continue, let's set up some default settings to apply to our Book in the following examples

In [96]:
# Applying these at the Book level will affect every cell in every sheet
book_settings = dict(
    row_height=29,
    col_width=5, # same dimensions as rows. Square cells.
    center=True, # sets align_x='center' and align_y='center'
    bold=True,
    size=14,  # font
    auto_open=True,  # opens our book for us
    zoom=350,
    end_gap=True,
    # Note: end_gap surrounds the sheet's contents with Cells.
    # (True applies a default). This way the row_height
    # and col_width are applied everywhere
)

In [ ]:
theme = xb.colors.theme
# Note: theme colors are numbered in ascending order of darkness.
# green4 is darker, and green1 is lighter
Book(
    Sheet(
        Frame(
            Col(
                Cell(1, fill_color=theme.green4),  # Cell-level
                Cell(2),
                fill_color=theme.green3,  # Col-level
                size=11,
            ),
            Col(3, 4),
            fill_color=theme.green2,  # Frame-level
        ),
    ),
    **book_settings
).write(PATH)

<img src="https://i.imgur.com/YqtdwEu.png" width="150"/>

## Colors

For the next few examples I'll use ``xb.colors.theme.groups``, each color in 'groups' contains a list of 6 shades, as seen in the default color picker you use every time you open Excel.

In [ ]:
reds = theme.groups.red[1:] # last 5 colors.
blues = theme.groups.light_blue[1:]
Book(
    Sheet(
        Row([Cell(i+1, fill_color=c) for i,c in enumerate(reds)]),
        Row([Cell(i+1, fill_color=c) for i,c in enumerate(blues)]),
    ),
    **book_settings
).write(PATH)

<img src="https://i.imgur.com/4wgk1db.png" width="350"/>

``auto_color_font`` ensures the text is always readable

In [ ]:
Book(
    Sheet(
        Row([Cell(i+1, fill_color=c) for i,c in enumerate(reds)]),
        Row([Cell(i+1, fill_color=c) for i,c in enumerate(blues)]),
        auto_color_font=True,
    ),
    **book_settings
).write(PATH)

<img src="https://i.imgur.com/GiM2HTA.png" width="350"/>

It gets better: ``auto_shade_font`` will take your background color and give the font a *scaled* version of it.

In [ ]:
Book(
    Sheet(
        Row([Cell(i+1, fill_color=c) for i,c in enumerate(reds)]),
        Row([Cell(i+1, fill_color=c) for i,c in enumerate(blues)]),
        auto_shade_font=True,
    ),
    **book_settings,
).write(PATH)

<img src="https://i.imgur.com/wvJkN4k.png" width="350"/>

## Headers

Series objects - ``Col`` and ``Row`` - have a special attribute: ``header``.

A header is a string defined as an attribute on an element, that won't be converted
to a Cell until write time. Separating it from the series' children serves a few purposes:

- It's ignored when the series is used in an arithmetic expression or formula (more on that later)
- Its styling is independent from the rest of the series' Cells.
- Convenience when passing an existing data source (like a list, or pandas Series) as the series' data

In [105]:
book_settings = dict(
    row_height=29,
    col_width=5,
    center=True,
    auto_open=True,
    zoom=350,
    end_gap=True,
)

In [ ]:
Book(
    Sheet(
        Row(1,2,3, header="one", fill_color=theme.red2),
    ),
    **book_settings,
).write(PATH)

<img src="https://i.imgur.com/pKjGFpO.png" width="300"/>

Col and Row have a ``header_style`` attribute

In [ ]:
Book(
    Sheet(
        Col(
            Cell(1),
            Cell(2),
            Cell(3),
            header="one",
            fill_color=theme.purple1,
            header_style=dict(
                center=True,
                fill_color=theme.purple4,
                bold=True,
                auto_color_font=True,
            )
        ),
    ),
    **book_settings,
).write(PATH)

<img src="https://i.imgur.com/sBxeZVr.png" width="75"/>

## Frames

Col and Row will use the series name of a given pandas DataFrame as their header automatically.
Consequently, if a `Frame` is given a DataFrame, it will use the column names as headers.

In [127]:
import pandas as pd

df = pd.DataFrame(
    zip([1,2,3,4], ['one', 'two', 'three', 'four']),
    columns=['Number', 'Word']
)

fr = xb.Frame(df)
fr

Number,Word
Cell(1),Cell(one)
Cell(2),Cell(two)
Cell(3),Cell(three)
Cell(4),Cell(four)


`VFrame` is Frame's sibling. It holds rows and arranges them vertically

In [118]:
vfr = xb.VFrame(df)
vfr

,,,,
Number,Cell(1),Cell(2),Cell(3),Cell(4)
Word,Cell(one),Cell(two),Cell(three),Cell(four)
,,,,


We can access elements in a Frame by their header

In [119]:
fr['Word']

Word
Cell(one)
Cell(two)
Cell(three)
Cell(four)
""


In [120]:
vfr['Word']

,,,,
Word,Cell(one),Cell(two),Cell(three),Cell(four)
,,,,


## Python Expressions and Cell References

Excelbird will track 'cell references' on its own, and later convert them to real cell locations when the Book is written

In [128]:
fr['Big Number'] = fr['Number'] + " thousand"
fr

Number,Word,Big Number
Cell(1),Cell(one),Cell({...})
Cell(2),Cell(two),Cell({...})
Cell(3),Cell(three),Cell({...})
Cell(4),Cell(four),Cell({...})


In [ ]:
xb.Book(fr, auto_open=True).write(PATH)

<img src="https://i.imgur.com/PuxeEGQ.png" width="300"/>

In [ ]:
from excelbird import fn  # All 506 builtin excel functions here
fr = xb.Frame(df)
fr['Big Number'] = fr['Number'] + " thousand"

fr['Combined'] = fn.CONCAT( fr.loc['Word':'Big Number'] )

xb.Book(fr, auto_open=True).write(PATH)

<img src="https://i.imgur.com/XOXLZbv.png" width="400"/>

More coming soon!